# Entrega 6 - Extração de Dados Estruturados
## Ciência de Dados Aplicada ao Direito II

**Objetivo:** Extrair informações estruturadas dos casos de crédito consignado (já filtrados pela Entrega 5)

### Fluxo de Trabalho:
1. **Entrega 5** → Filtra casos de crédito consignado do dataset completo
2. **Entrega 6** → Extrai informações detalhadas APENAS dos casos filtrados

### Campos Extraídos:
- `cd_atendimento`: ID do caso
- `nome_empresa`: Nome da empresa no polo passivo
- `cnpj`: CNPJ válido (14 dígitos)
- `valor_causa`: Valor da causa em reais
- `dt_distribuicao`: Data de distribuição (YYYY-MM-DD)
- `tipo_vara`: JE (Juizado Especial) ou G1 (Vara Comum)
- `uf`: Unidade Federativa

In [19]:
# Imports
import re
import unicodedata
import pandas as pd
from typing import List, Tuple

print("Bibliotecas importadas com sucesso")

Bibliotecas importadas com sucesso


## 1. Carregamento de Dados

Carrega o dataset completo e filtra apenas os casos de crédito consignado identificados pela Entrega 5.

In [20]:
# Carrega o dataset COMPLETO
INPUT_CSV = "../data/dataset_clinica20252.csv"
df_completo = pd.read_csv(INPUT_CSV, sep="|")

print("="*80)
print("CARREGAMENTO DE DADOS")
print("="*80)
print(f"\nDataset completo: {df_completo.shape[0]} linhas x {df_completo.shape[1]} colunas")

# Carrega os cd_atendimento FILTRADOS da Entrega 5
ENTREGA5_OUTPUT = "../entrega5/output.xlsx"

try:
    df_filtrados = pd.read_excel(ENTREGA5_OUTPUT)
    print(f"\nCasos de crédito consignado (Entrega 5): {len(df_filtrados)}")
    
    # Filtra o dataset completo
    cd_list = df_filtrados['cd_atendimento'].astype(str).tolist()
    df = df_completo[df_completo['cd_atendimento'].astype(str).isin(cd_list)].copy()
    
    print(f"Dataset filtrado: {df.shape[0]} linhas x {df.shape[1]} colunas")
    print(f"\n✓ Dataset pronto para extração!")
    
except FileNotFoundError:
    print(f"\n⚠️  AVISO: Arquivo da Entrega 5 não encontrado!")
    print(f"Esperado: {ENTREGA5_OUTPUT}")
    print("Usando dataset completo (resultados podem não estar corretos)")
    df = df_completo.copy()

CARREGAMENTO DE DADOS

Dataset completo: 19800 linhas x 6 colunas

Casos de crédito consignado (Entrega 5): 8556
Dataset filtrado: 8557 linhas x 6 colunas

✓ Dataset pronto para extração!

Casos de crédito consignado (Entrega 5): 8556
Dataset filtrado: 8557 linhas x 6 colunas

✓ Dataset pronto para extração!


## 2. Funções Auxiliares

Funções de limpeza e normalização de texto.

In [21]:
def strip_accents(s: str) -> str:
    """Remove acentos de uma string."""
    if not isinstance(s, str): 
        return ""
    return "".join(c for c in unicodedata.normalize("NFD", s) 
                   if unicodedata.category(c) != "Mn")

def clean_text(s: str) -> str:
    """Limpa e normaliza texto."""
    s = (s or "").replace("\n", " ")
    return re.sub(r"\s+", " ", s).strip()

def limpar_para_excel(texto):
    """Remove caracteres de controle que causam problemas no Excel."""
    if not isinstance(texto, str):
        return texto
    # Remove caracteres de controle (0x00-0x1F exceto tab, newline, carriage return)
    return re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F]', '', texto)

# Constantes
UF_LIST = set("AC AL AP AM BA CE DF ES GO MA MT MS MG PA PB PR PE PI RJ RN RS RO RR SC SP SE TO".split())

print("✓ Funções auxiliares definidas")

✓ Funções auxiliares definidas


## 3. Extração de CNPJ

Validação de CNPJs e busca com validação de dígitos verificadores.

In [22]:
def cnpj_is_valid(cnpj_digits: str) -> bool:
    """Valida dígitos verificadores do CNPJ (14 dígitos)."""
    if len(cnpj_digits) != 14 or len(set(cnpj_digits)) == 1:
        return False
    
    nums = [int(x) for x in cnpj_digits]
    
    # Valida os dois dígitos verificadores
    for i in [12, 13]:
        pesos = [5,4,3,2,9,8,7,6,5,4,3,2] if i == 12 else [6,5,4,3,2,9,8,7,6,5,4,3,2]
        soma = sum(a * b for a, b in zip(nums[:i], pesos))
        dig = 11 - (soma % 11)
        dig = 0 if dig >= 10 else dig
        if nums[i] != dig: 
            return False
    
    return True


def find_cnpjs_pos(text: str) -> List[Tuple[str, int]]:
    """Encontra CNPJs válidos no texto com suas posições."""
    out = []
    pattern = r"\b\d{2}\.?\d{3}\.?\d{3}/?\d{4}-?\d{2}\b"
    
    for m in re.finditer(pattern, text):
        digits = re.sub(r"\D", "", m.group(0))
        if len(digits) == 14 and cnpj_is_valid(digits):
            out.append((digits, m.start()))
    
    return out


print("✓ Funções de validação de CNPJ definidas")

✓ Funções de validação de CNPJ definidas


## 4. Extração de Nome de Empresa

Estratégia simplificada sem regex complexa:
1. Procura "BANCO" ou sufixo societário (S.A, LTDA, etc.)
2. Captura APENAS palavras com letras
3. Para quando encontrar: números, CNPJ, endereço, etc.

In [23]:
def extract_company_name_simple(text: str) -> Tuple[str, int, int]:
    """
    Extrai nome da empresa de forma simplificada.
    
    Retorna: (nome_empresa, posicao_inicio, posicao_fim) ou ("vazio", -1, -1)
    """
    T = clean_text(text).upper()
    
    # Palavras que indicam FIM do nome da empresa
    STOP_WORDS = {
        'INSCRITO', 'INSCRITA', 'CNPJ', 'CPF', 'SITO', 'SITA',
        'ENDERECO', 'ENDEREÇO', 'RUA', 'AVENIDA', 'PRACA', 'PRAÇA',
        'NUMERO', 'NÚMERO', 'CEP', 'BAIRRO', 'CIDADE', 'ESTADO',
        'REPRESENTADO', 'REPRESENTADA', 'ADVOGADO', 'ADVOGADA',
        'QUALIFICADO', 'QUALIFICADA', 'BRASILEIRO', 'BRASILEIRA'
    }
    
    # Sufixos societários válidos
    SUFFIXES = ['S.A.', 'S.A', 'S/A', 'LTDA', 'LTDA.', 'EIRELI', 'ME', 'EPP']
    
    # Procura "BANCO" no texto
    banco_pos = T.find('BANCO')
    if banco_pos == -1:
        # Se não tem BANCO, procura por sufixo societário
        for suffix in SUFFIXES:
            if suffix in T:
                suffix_pos = T.find(suffix)
                start = max(0, suffix_pos - 100)
                trecho = T[start:suffix_pos + len(suffix)]
                words = trecho.split()[-6:]
                nome = ' '.join(words).strip()
                return (nome, start, suffix_pos + len(suffix))
        
        return ("vazio", -1, -1)
    
    # A partir de "BANCO", captura palavras seguintes
    start_pos = banco_pos
    trecho = T[banco_pos:]
    palavras = trecho.split()
    
    nome_parts = []
    last_pos = banco_pos
    
    for i, palavra in enumerate(palavras):
        palavra_limpa = palavra.strip('.,;:()')
        
        # PARA se encontrar palavra-chave de parada
        if palavra_limpa in STOP_WORDS:
            break
        
        # PARA se encontrar números (exceto em sufixos)
        if any(char.isdigit() for char in palavra_limpa):
            if not any(s in palavra for s in ['S.A', 'S/A']):
                break
        
        # PARA se tiver mais de 6 palavras
        if i > 6:
            break
        
        nome_parts.append(palavra)
        last_pos = banco_pos + trecho.find(palavra) + len(palavra)
        
        # PARA se encontrou sufixo societário
        if any(s in palavra for s in SUFFIXES):
            break
    
    if nome_parts:
        nome = ' '.join(nome_parts).strip('.,;: ')
        return (nome, start_pos, last_pos)
    
    return ("vazio", -1, -1)


def find_company_spans(text: str) -> List[Tuple[str, int, int]]:
    """Retorna lista com apenas 1 empresa (a primeira encontrada)."""
    nome, start, end = extract_company_name_simple(text)
    
    if nome != "vazio":
        return [(nome, start, end)]
    
    return []


print("✓ Funções de extração de empresa definidas")

✓ Funções de extração de empresa definidas


## 5. Associação Empresa + CNPJ

Retorna APENAS 1 CNPJ por caso, priorizando proximidade ao nome da empresa.

In [24]:
def pick_company_and_cnpjs_v3(text: str, win_after=400, win_before=200) -> Tuple[str, str]:
    """
    Extrai empresa e CNPJ com proximidade.
    
    - Retorna APENAS 1 CNPJ (não uma lista)
    - Prioriza CNPJ mais próximo ao nome da empresa
    - Janela: 400 chars depois, 200 antes
    
    Retorna: (nome_empresa, cnpj_string)
    """
    T = clean_text(text)
    companies = find_company_spans(T)
    cnpjs_pos = find_cnpjs_pos(T)
    
    def cnpj_mais_proximo(start, end):
        """Retorna o CNPJ MAIS PRÓXIMO ao span da empresa."""
        candidatos = []
        for cnpj, pos in cnpjs_pos:
            if (start - win_before) <= pos <= (end + win_after):
                # Calcula distância (prioriza CNPJs após o nome)
                if pos >= start:
                    distancia = pos - end
                else:
                    distancia = (start - pos) * 2  # Penaliza CNPJs antes
                
                candidatos.append((cnpj, distancia))
        
        if candidatos:
            candidatos.sort(key=lambda x: x[1])
            return candidatos[0][0]
        
        return None
    
    # 1) Tenta associar por proximidade
    for name, s, e in companies:
        cnpj_proximo = cnpj_mais_proximo(s, e)
        if cnpj_proximo:
            return name, cnpj_proximo
    
    # 2) Fallback: retorna PRIMEIRO CNPJ válido
    if cnpjs_pos:
        primeiro_cnpj = cnpjs_pos[0][0]
        if companies:
            return companies[0][0], primeiro_cnpj
        return "vazio", primeiro_cnpj
    
    return "vazio", "vazio"


print("✓ Função de associação empresa+CNPJ definida")

✓ Função de associação empresa+CNPJ definida


## 6. Extração de Valor da Causa

Procura valores no formato brasileiro (1.234,56) próximos ao label "valor da causa".

In [25]:
def extract_valor_causa(text: str) -> float:
    """Extrai valor da causa em reais."""
    T = clean_text(text)
    t_lower = strip_accents(T).lower()
    
    # Limites razoáveis para ações de crédito consignado
    MIN_VALOR = 100.0
    MAX_VALOR = 10_000_000.0
    
    # Padrão: valores no formato brasileiro (1.234,56 ou R$ 1.234,56)
    pattern = r"R?\$?\s*(\d{1,3}(?:\.\d{3})*,\d{2}|\d+,\d{2})"
    
    # 1) Primeiro tenta próximo a "valor da causa"
    if "valor" in t_lower and "causa" in t_lower:
        pos = t_lower.find("valor")
        trecho = T[max(0, pos-150):pos+150]
        
        valores = []
        for match in re.finditer(pattern, trecho):
            num_str = match.group(1)
            valor = float(num_str.replace(".", "").replace(",", "."))
            if MIN_VALOR <= valor <= MAX_VALOR:
                valores.append(valor)
        
        if valores:
            return max(valores)
    
    # 2) Busca em todo texto
    valores = []
    for match in re.finditer(pattern, T):
        num_str = match.group(1)
        valor = float(num_str.replace(".", "").replace(",", "."))
        if MIN_VALOR <= valor <= MAX_VALOR:
            valores.append(valor)
    
    return max(valores) if valores else 0.0


print("✓ Função de extração de valor definida")

✓ Função de extração de valor definida


## 7. Extração de Data de Distribuição

Procura data próxima a "distribuição" ou "autuação" em formatos DD/MM/YYYY ou YYYY-MM-DD.

In [26]:
def extract_dt_distribuicao(text: str) -> str:
    """Extrai data de distribuição no formato YYYY-MM-DD."""
    t = strip_accents(clean_text(text)).lower()
    
    # Padrões de data
    pattern_iso = r"(20\d{2})-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])"
    pattern_br = r"(0[1-9]|[12]\d|3[01])[/\-](0[1-9]|1[0-2])[/\-](20\d{2})"
    
    # 1) Procura próximo a "distribuição" ou "autuação"
    for keyword in ['distribuicao', 'distribuido', 'autuacao', 'autuado']:
        pos = t.find(keyword)
        if pos != -1:
            trecho = t[max(0, pos-100):pos+100]
            
            # Tenta ISO
            match = re.search(pattern_iso, trecho)
            if match:
                return f"{match.group(1)}-{match.group(2)}-{match.group(3)}"
            
            # Tenta BR
            match = re.search(pattern_br, trecho)
            if match:
                dd, mm, y = match.group(1), match.group(2), match.group(3)
                return f"{y}-{mm.zfill(2)}-{dd.zfill(2)}"
    
    # 2) Fallback: primeira data ISO
    match = re.search(pattern_iso, t)
    if match:
        return f"{match.group(1)}-{match.group(2)}-{match.group(3)}"
    
    # 3) Fallback: primeira data BR
    match = re.search(pattern_br, t)
    if match:
        dd, mm, y = match.group(1), match.group(2), match.group(3)
        return f"{y}-{mm.zfill(2)}-{dd.zfill(2)}"
    
    return ""


print("✓ Função de extração de data definida")

✓ Função de extração de data definida


## 8. Extração de Tipo de Vara e UF

Funções para classificar tipo de vara e extrair UF.

In [27]:
def classify_tipo_vara(text: str) -> str:
    """Classifica o tipo de vara: JE (Juizado Especial) ou G1 (Vara Comum)."""
    t = strip_accents(clean_text(text)).lower()
    if "juizado especial" in t or "jecc" in t:
        return "JE"
    return "G1"


def extract_uf(text: str) -> str:
    """Extrai a UF (estado) do texto."""
    tokens = re.findall(r"\b[A-Z]{2}\b", clean_text(text).upper())
    for tk in tokens:
        if tk in UF_LIST:
            return tk
    return ""


print("✓ Funções de tipo de vara e UF definidas")

✓ Funções de tipo de vara e UF definidas


## 9. Função Principal de Extração

Processa todos os casos e extrai os 7 campos estruturados.

In [28]:
def extrair_dados(df: pd.DataFrame, output_path: str = "output.xlsx") -> pd.DataFrame:
    """
    Extrai todos os campos de cada caso.
    
    Retorna: DataFrame com os dados extraídos + salva em Excel
    """
    print("="*80)
    print("INICIANDO EXTRAÇÃO DE DADOS")
    print("="*80)
    
    # Carrega colunas de texto
    qual = df.get("ds_Qualificacao", "").astype(str)
    fatos = df.get("ds_fatos", "").astype(str)
    pedidos = df.get("ds_Pedidos", "").astype(str)
    acao = df.get("ds_Acao_Judicial", "").astype(str)
    
    # Função para extrair empresa e CNPJ
    def extrair_cnpj_empresa(row):
        # Prioriza ds_Qualificacao (polo passivo)
        nome, cnpj = pick_company_and_cnpjs_v3(row['ds_Qualificacao'])
        
        if cnpj != "vazio":
            return nome, cnpj
        
        # Fallback: busca em todas as colunas
        texto_completo = f"{row['ds_Qualificacao']} {row['ds_fatos']} {row['ds_Pedidos']} {row['ds_Acao_Judicial']}"
        nome, cnpj = pick_company_and_cnpjs_v3(texto_completo)
        
        return nome, cnpj
    
    # Extrai nome da empresa e CNPJ
    print("\n1/7 Extraindo empresa e CNPJ...")
    df_temp = df[['ds_Qualificacao', 'ds_fatos', 'ds_Pedidos', 'ds_Acao_Judicial']].fillna('')
    pares = df_temp.apply(extrair_cnpj_empresa, axis=1)
    nome_empresa = pares.map(lambda x: x[0])
    cnpjs = pares.map(lambda x: x[1])
    
    # Prepara textos para outras extrações
    texto_valor = (fatos + " " + pedidos).astype(str)
    texto_data = (pedidos + " " + qual + " " + acao).astype(str)
    
    # Extrai outros campos
    print("2/7 Extraindo valor da causa...")
    valores = texto_valor.map(extract_valor_causa)
    
    print("3/7 Extraindo data de distribuição...")
    datas = texto_data.map(extract_dt_distribuicao)
    
    print("4/7 Classificando tipo de vara...")
    tipos_vara = qual.map(classify_tipo_vara)
    
    print("5/7 Extraindo UF...")
    ufs = qual.map(extract_uf)
    
    # Cria DataFrame
    print("6/7 Montando DataFrame...")
    resultado = pd.DataFrame({
        "cd_atendimento": df["cd_atendimento"].astype(str),
        "nome_empresa": nome_empresa,
        "cnpj": cnpjs,
        "valor_causa": valores,
        "dt_distribuicao": datas,
        "tipo_vara": tipos_vara,
        "uf": ufs,
    })
    
    # Limpa caracteres problemáticos para Excel
    for col in ["nome_empresa", "cnpj", "dt_distribuicao", "tipo_vara", "uf"]:
        resultado[col] = resultado[col].map(limpar_para_excel)
    
    # Salva resultado
    print(f"7/7 Salvando arquivo '{output_path}'...")
    resultado.to_excel(output_path, index=False)
    
    print(f"\n✓ Arquivo '{output_path}' criado com sucesso!")
    print("="*80)
    
    return resultado


print("✓ Função principal definida")

✓ Função principal definida


## 10. Execução

Execute a extração e visualize os resultados.

In [ ]:
# Executa a extração
resultado = extrair_dados(df, output_path="output.xlsx")

# Estatísticas
print("\n" + "="*80)
print("ESTATÍSTICAS")
print("="*80)
print(f"\nTotal de casos processados: {len(resultado)}")

print(f"\n Distribuição por UF (top 10):")
print(resultado['uf'].value_counts().head(10))

print(f"\n Distribuição por Tipo de Vara:")
print(resultado['tipo_vara'].value_counts())

print(f"\n CNPJs extraídos:")
cnpjs_validos = len(resultado[resultado['cnpj'] != 'vazio'])
print(f"  Válidos: {cnpjs_validos} ({cnpjs_validos/len(resultado)*100:.1f}%)")
print(f"  Vazios: {len(resultado) - cnpjs_validos}")

print(f"\n Valores da causa:")
valores_validos = len(resultado[resultado['valor_causa'] > 0])
print(f"  Extraídos: {valores_validos} ({valores_validos/len(resultado)*100:.1f}%)")
print(f"  Média: R$ {resultado[resultado['valor_causa'] > 0]['valor_causa'].mean():,.2f}")

print(f"\n Datas extraídas:")
datas_validas = len(resultado[resultado['dt_distribuicao'] != ''])
print(f"  Válidas: {datas_validas} ({datas_validas/len(resultado)*100:.1f}%)")

print("\n" + "="*80)
print("AMOSTRA DOS RESULTADOS (primeiras 10 linhas)")
print("="*80)
display(resultado.head(10))

INICIANDO EXTRAÇÃO DE DADOS

1/7 Extraindo empresa e CNPJ...
2/7 Extraindo valor da causa...
3/7 Extraindo data de distribuição...
4/7 Classificando tipo de vara...
5/7 Extraindo UF...
6/7 Montando DataFrame...
7/7 Salvando arquivo 'output.xlsx'...

✓ Arquivo 'output.xlsx' criado com sucesso!

ESTATÍSTICAS

Total de casos processados: 8557

📊 Distribuição por UF (top 10):
uf
PI    1504
MA    1322
SP    1012
BA     682
CE     502
MG     435
AM     345
SC     334
AL     239
RJ     236
Name: count, dtype: int64

📊 Distribuição por Tipo de Vara:
tipo_vara
G1    6846
JE    1711
Name: count, dtype: int64

📊 CNPJs extraídos:
  Válidos: 7743 (90.5%)
  Vazios: 814

📊 Valores da causa:
  Extraídos: 8522 (99.6%)
  Média: R$ 18,444.90

📊 Datas extraídas:
  Válidas: 4514 (52.8%)

AMOSTRA DOS RESULTADOS (primeiras 10 linhas)


,cd_atendimento,nome_empresa,cnpj,valor_causa,dt_distribuicao,tipo_vara,uf
35,0800194-50.2025.8.10.0126,BANCO BRADESCO S.A,60746948000112,14992.00,2025-02-07,G1,TO
59,1009294-25.2025.8.26.0506,BANCO BRADESCO S/A,60746948000112,10100.19,,G1,SP
65,1017110-70.2025.8.26.0405,BANCO BRADESCO S.A,60746948000112,542.64,,G1,SP
75,0002561-03.2025.8.16.0130,vazio,vazio,10000.00,,G1,PR
82,0800681-57.2025.8.18.0074,vazio,vazio,40255.40,2025-04-10,G1,MS
96,0801193-31.2025.8.18.0077,BANCO BRADESCO S.A,60746948000112,145.90,2025-06-12,G1,PI
99,0801915-71.2024.8.10.0126,BANCO BRADESCO,60746948000112,10000.00,2024-10-30,G1,MA
156,0141882-95.2025.8.04.1000,"BANCO BRADESCO., PESSOA JURÍDICA DE DIREITO PR...",60746948000112,19950.00,,G1,AM
187,0702310-89.2025.8.02.0046,BANCO BRADESCO S.A,60746948061251,299.90,,G1,AL
197,0800458-85.2025.8.18.0048,DE BRADESCO VIDA E PREVIDÊNCIA S/A,51990695000137,15000.00,2025-03-06,G1,PI
